# Composio

> [Composio](https://composio.dev) is an integration platform that provides 250+ production-ready tools for AI agents with flexible authentication management.

## Installation and Usage

In [1]:
%pip install composio-langchain langchain-openai

after the installation is complete, either run `composio login` or export your composio API key as `COMPOSIO_API_KEY`. Get your Composio API from [here](https://app.composio.dev)

## Example

In this example, we will use Composio's **GitHub** tool to star a repository on GitHub.

### 1. Initialize Composio toolset & LLM

In [2]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain import hub
from langchain_openai import ChatOpenAI
from composio_langchain import ComposioToolSet, App, Action

toolset = ComposioToolSet()
llm = ChatOpenAI()

### 2. Connect your GitHub account

Using the CLI:

In [3]:
%composio add github

Using the Python SDK:

In [ ]:
request = toolset.initiate_connection(app=App.GITHUB)
print(f"Open this URL to authenticate: {request.redirectUrl}")

### 3. Get Tools

- Retrieving all the tools from an app (not recommended for production):

In [ ]:
tools = toolset.get_tools(apps=[App.GITHUB])

- Filtering tools based on **tags**:

In [ ]:
tag = "users"

filtered_action_enums = toolset.find_actions_by_tags(
    App.GITHUB,
    tags=[tag], 
)

tools = toolset.get_tools(actions=filtered_action_enums)

- Filtering tools based on **use case**:

In [ ]:
use_case = "Star a repository on GitHub"

filtered_action_enums = toolset.find_actions_by_use_case(
    App.GITHUB, use_case=use_case, advanced=False
)

tools = toolset.get_tools(actions=filtered_action_enums)

Set the `advanced` to True to get actions for complex use cases

- Using specific tools, In this demo, we will use the `GITHUB_STAR_A_REPOSITORY_FOR_THE_AUTHENTICATED_USER` action from the GitHub app:

In [ ]:
tools = toolset.get_tools(
    actions=[Action.GITHUB_STAR_A_REPOSITORY_FOR_THE_AUTHENTICATED_USER]
)

Learn more about filtering actions [here](https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions)

### 4. Define agent

In [ ]:
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### 5. Execute task

In [ ]:
task = "Star a repo composiohq/composio on GitHub"
agent_executor.invoke({"input": task})

More detailed list of tools can be found [here](https://app.composio.dev)